In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.cuda.amp import autocast, GradScaler
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
import plotly.colors as pc
import plotly.subplots as sp
import time
import pandas as pd


class Network(nn.Module):

    def __init__(self, width: int, depth: int, act, initialise = True):
        super().__init__()
        
        funcs = {"relu": nn.ReLU, 
                 "selu": nn.SELU, 
                 "tanh": nn.Tanh, 
                 "sigmoid": nn.Sigmoid, 
                 "leakyrelu": nn.LeakyReLU}

        # activation function
        activation = funcs[act]
        
        
        # input layer
        self.input = nn.Linear(3, depth)
        if initialise:
            init.kaiming_normal_(self.input.weight,nonlinearity=act)

        # hidden layers
        self.hidden = nn.Sequential(
            *[
                nn.Sequential(nn.Linear(depth, depth), activation())
                for _ in range(width - 2)
            ]
        )

        for layer in self.hidden.children():
            if isinstance(layer, nn.Linear) and initialise:
                init.kaiming_normal_(layer.weight,nonlinearity=act)

        # output layer
        self.output = nn.Linear(depth, 1)

        self.constants()

    def constants(self) -> None:
        self.L: float = 0.1  # m
        self.sigma: float = 0.5

        self.k: int = 15  # W/mK

        self.ht: int = 200
        self.hb: int = 100

        self.Bi_t: float = self.ht * self.L / self.k
        self.Bi_b: float = self.hb * self.L / self.k

        self.lossTotal: list[float] = [1]
        self.epochData: dict[int, np.ndarray] = {}
        
        self.Xpoints: int = 21
        self.Ypoints: int = 11
        self.Tpoints: int = 6
        
        self.Tmax: float = 5.0

    def forward(
        self, x: torch.Tensor, y: torch.Tensor, t: torch.Tensor
    ) -> torch.Tensor:
        inp = torch.cat((x, y, t), dim=1)
        inputted = self.input(inp)
        hidden = self.hidden(inputted)
        out = self.output(hidden)
        return out

    def init_boundaries(self, N: int) -> None:

        # Defines a vertical tensor of -sigma and sigma y values, then extends the vector to a 3D tensor for
        # boundary training then reshapes it to 4D column tensor for inputting into the network
        self.y0_boundary: torch.Tensor = (
            torch.full((N, 1), -self.sigma)
            .expand(N, N, N)
            .reshape(N**3, 1)
            .requires_grad_(True)
            .to(device)
        )
        self.y1_boundary: torch.Tensor = (
            torch.full((N, 1), self.sigma)
            .expand(N, N, N)
            .reshape(N**3, 1)
            .requires_grad_(True)
            .to(device)
        )

        # Same operations as above but for random values in the range of -sigma and sigma for initial condition
        self.y_rand: torch.Tensor = (
            (-2 * self.sigma * torch.rand(N, 1) + self.sigma)
            .expand(N, N, N)
            .reshape(N**3, 1)
            .requires_grad_(True)
            .to(device)
        )

        # Defines a vertical tensor of -1 and 1 x values, then extends the vector to a 3D tensor for
        # boundary training then reshapes it to 4D column tensor for inputting into the network
        self.x0_boundary: torch.Tensor = (
            torch.full((N,), -1.0)
            .expand(N, N, N)
            .reshape(N**3, 1)
            .requires_grad_(True)
            .to(device)
        )
        self.x1_boundary: torch.Tensor = (
            torch.full((N,), 1.0)
            .expand(N, N, N)
            .reshape(N**3, 1)
            .requires_grad_(True)
            .to(device)
        )

        # Same operations as above but for random values in the range of -1 and 1 for initial condition
        self.x_rand: torch.Tensor = (
            (-2 * torch.rand(N) + 1.0)
            .expand(N, N, N)
            .reshape(N**3, 1)
            .requires_grad_(True)
            .to(device)
        )

        # Defines a vertical tensor of 0 t values, then extends the vector to a 3D tensor for
        # initial condition training then reshapes it to 4D column tensor for inputting into the network
        self.t0_boundary: torch.Tensor = (
            torch.tensor(0.0)
            .expand(N, N, N)
            .reshape(N**3, 1)
            .requires_grad_(True)
            .to(device)
        )

        # Same operations as above but for random values in the range of 0 and 10 for boundary training
        self.t_rand: torch.Tensor = (
            (self.Tmax*torch.rand(N, 1, 1))
            .expand(N, N, N)
            .reshape(N**3, 1)
            .requires_grad_(True)
            .to(device)
        )

    def derivative(self, f: torch.Tensor, x: torch.Tensor) -> torch.Tensor:
        return torch.autograd.grad(
            f, x, grad_outputs=torch.ones_like(f).to(device), create_graph=True, allow_unused= True
        )[0]

    def losses(self, N: int) -> torch.Tensor:

        self.init_boundaries(N)

        # -1x all y values boundary condition is cooling heat transfer in the y direction
        T: torch.Tensor = self.forward(self.x0_boundary, self.y_rand, self.t_rand)
        T_x0: torch.Tensor = self.derivative(T, self.x0_boundary)

        minusx_loss: torch.Tensor = torch.mean((T_x0 - self.Bi_b * T) ** 2)

        # 1x all y values boundary condition is heating heat transfer in the y direction
        T2: torch.Tensor = self.forward(self.x1_boundary, self.y_rand, self.t_rand)
        T_x1: torch.Tensor = self.derivative(T2, self.x1_boundary)

        x_loss: torch.Tensor = torch.mean(
            (T_x1 + self.Bi_t * (T2 - 1)) ** 2
        )

        # -sigma y all x values boundary condition is adiabatic in the x direction
        T3: torch.Tensor = self.forward(self.x_rand, self.y0_boundary, self.t_rand)
        T_y0: torch.Tensor = self.derivative(T3, self.y0_boundary)

        minusy_loss: torch.Tensor = torch.mean(T_y0**2)

        # sigma y all x values boundary condition is adiabatic in the x direction
        T4: torch.Tensor = self.forward(self.x_rand, self.y1_boundary, self.t_rand)
        T_y1: torch.Tensor = self.derivative(T4, self.y1_boundary)

        y_loss: torch.Tensor = torch.mean(T_y1**2)

        # loss for initial condition

        T_IC: torch.Tensor = self.forward(self.x_rand, self.y_rand, self.t0_boundary)
        t0_loss: torch.Tensor = torch.mean((T_IC - 1) ** 2)

        # loss for physics sample
        T_phy: torch.Tensor = self.forward(self.x_rand, self.y_rand, self.t_rand)
        
        T_x_phy: torch.Tensor = self.derivative(T_phy, self.x_rand)
        T_xx_phy: torch.Tensor = self.derivative(T_x_phy, self.x_rand)
        
        T_y_phy: torch.Tensor = self.derivative(T_phy, self.y_rand)
        T_yy_phy: torch.Tensor = self.derivative(T_y_phy, self.y_rand)
        
        T_t_phy: torch.Tensor = self.derivative(T_phy, self.t_rand)

        phys_loss: torch.Tensor = torch.mean(
            (T_t_phy - 4 * (T_xx_phy + (T_yy_phy / (self.sigma**2) )) ) ** 2
        )

        loss: torch.Tensor = (
            minusx_loss + x_loss + minusy_loss + y_loss + t0_loss + phys_loss
        )
        return loss

    def train(
        self, epochs_max: int, N: int, optimiser: str = "LBFGS", lr: float = 1
    ) -> None:

        def closure() -> float:
            optimizer.zero_grad()
            loss: torch.Tensor = self.losses(N)
            loss.backward()
            torch.nn.utils.clip_grad_value_(self.parameters(), clip_value=1.0)
            return loss

        loss_val: float = self.lossTotal[-1]
        epoch: int = len(self.lossTotal) - 1
        epoch_times: list[float] = []
        window_size: int = 10

        scaler = GradScaler()

        if optimiser == "Adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        elif optimiser == "LBFGS":
            optimizer = torch.optim.LBFGS(
                self.parameters(),
                max_iter=100,
                history_size=100,
                line_search_fn="strong_wolfe",
            )
        else:
            raise ValueError("Invalid optimiser")

        start: float = time.time()

        while epoch < epochs_max and loss_val > 1e-6:
            epoch += 1
            if optimiser == "Adam":
                optimizer.zero_grad()
                with torch.cuda.amp.autocast():
                    loss: torch.Tensor = self.losses(N)

                scaler.scale(loss).backward()
                scaler.unscale_(optimizer)  # unscale the gradients of optimizer's assigned params in-place
                torch.nn.utils.clip_grad_value_(
                    self.parameters(), clip_value=1.0
                )  # clip the unscaled gradients
                scaler.step(optimizer)
                scaler.update()
            else:
                loss: torch.Tensor = optimizer.step(closure)

            loss_val: float = loss.item()
            self.lossTotal.append(loss_val)

            # compute epoch time
            epoch_time: float = time.time() - start
            epoch_times.append(epoch_time)
            start: float = time.time()  # Reset the start time for the next epoch

            # Compute the moving average of epoch times
            window_times: list[float] = epoch_times[-window_size:]
            avg_epoch_time: float = sum(window_times) / len(window_times)

            # Calculate the remaining time
            time_remaining: float = (epochs_max - epoch) * avg_epoch_time

            # print loss and parameter values
            print(
                f'Epoch: {epoch}  Loss: {loss_val:.8f} Time/Epoch: {avg_epoch_time:.4f}s ETA: {time.strftime("%H Hours %M Minutes and %S Seconds", time.gmtime(time_remaining))}',
                end="\r",
            )
        
        self.time = epoch_times


In [3]:
import json
import gc
# Define the range of network sizes to explore
width_range = ["3","4","5","6","7","8"] 
depth_range = ["2", "4", "8", "16", "32", "64", "128", "256","512"] #1024 fills ram

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# nn seed
torch.manual_seed(123)

with open('size_loss.json') as f:
    size_loss_values = json.load(f)
    
with open('size_time.json') as f:
    size_time_values = json.load(f)

# Perform the parameter space study
for width in width_range:
    for depth in depth_range:
        torch.cuda.empty_cache()
        gc.collect()
        if size_loss_values.get(width) is not None and size_loss_values[width].get(depth) is not None:
            continue
        print(f"\nWidth: {width}  Depth: {depth}")
        # Create a new instance of the Network class with the specified width and depth
        pinn = Network(int(width), int(depth),'selu', True).to(device)
        
        # Train the network for 20000 epochs
        pinn.train(10000, 50, 'Adam', 1e-4)
        
        # Store the loss value in the dictionary
        
        size_loss_values[width][depth] = pinn.lossTotal
        size_time_values[width][depth] = pinn.time
        del pinn
        with open('size_loss.json', 'w') as f:
            json.dump(size_loss_values, f)
            
        with open('size_time.json', 'w') as f:
            json.dump(size_time_values, f)



Width: 7  Depth: 512


KeyboardInterrupt: 

In [19]:
import json

lossjson = {3: {depth: size_loss_values.get((3, depth),[]) for depth in depth_range } }
timejson = {3: {depth: size_time_values.get((3, depth),[]) for depth in depth_range } }

# Save size_loss_values dictionary as JSON
with open('size_loss.json', 'w') as file:

# Save size_time_values dictionary as JSON
with open('size_time.json', 'w') as file:
